# ⚽ Soccer Player Highlight Reel Generator - Production Grade

This notebook implements a complete, production-grade, end-to-end pipeline for generating personalized soccer player highlight reels. This is an advanced version with sophisticated algorithms for each stage.

## 🚀 Advanced Features
- **Player Detection**: High-performance YOLOv8 for person detection.
- **Multi-Object Tracking**: Full implementation of **ByteTrack** with a **Kalman Filter** for motion prediction and Hungarian Algorithm for association.
- **Long-term Re-Identification**: A hybrid system using a **Deep CNN** for appearance embeddings combined with color histograms and Jersey OCR.
- **Intelligent Event Detection**: A sophisticated heuristic model that analyzes player velocity, goal proximity, and clustering to identify key moments.
- **Professional Video Assembly**: Integration of **PySceneDetect** to find natural scene boundaries for clean clip extraction, stitched with FFmpeg.

## ⚡ How to Use
1. **Enable GPU**: Go to `Runtime` → `Change runtime type` and select `T4 GPU`.
2. **Run All Cells**: Click `Runtime` → `Run all`.
3. **Upload Video**: An upload prompt will appear. Select your soccer match video.
4. **Wait**: The pipeline will process the video. This is a computationally intensive process.
5. **Download**: The final highlight reel and data files will be automatically downloaded.

## 🔧 1. Setup & Installation

In [ ]:
# Install required packages (suppressing output for cleaner logs)
print("Installing dependencies...")
!pip install ultralytics torch torchvision opencv-python-headless easyocr scikit-learn numpy pandas tqdm pillow 'scenedetect[opencv]' --quiet

# Check GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ No GPU detected - using CPU (will be much slower)")

# Create project directories
import os
os.makedirs('/content/videos', exist_ok=True)
os.makedirs('/content/output', exist_ok=True)
os.makedirs('/content/temp_clips', exist_ok=True)

print("✅ Setup complete!")

## 📁 2. Upload Your Video

In [ ]:
from google.colab import files
import shutil

print("Please upload your soccer match video file.")
uploaded = files.upload()

video_path = None
for filename in uploaded.keys():
    if filename.lower().endswith(('.mp4', '.avi', '.mov')):
        destination_path = f'/content/videos/{filename}'
        shutil.move(filename, destination_path)
        print(f"✅ Video uploaded: {destination_path}")
        video_path = destination_path
        break

if not video_path:
    print("❌ No video file found. Please upload an MP4, AVI, or MOV file.")

## 🚀 3. Run the Full Pipeline

In [ ]:
# Import all required modules
import cv2
import torch
import numpy as np
import json
from ultralytics import YOLO
import easyocr
from tqdm.notebook import tqdm
import math
from sklearn.metrics.pairwise import cosine_similarity
import subprocess
import tempfile
from scipy.optimize import linear_sum_assignment
import torch.nn as nn
import torch.nn.functional as F
from typing import List, Dict, Tuple

# PySceneDetect imports
from scenedetect import open_video, SceneManager
from scenedetect.detectors import ContentDetector

print("✅ All modules imported successfully!")

In [ ]:
# Stage 1: Player Detection
print("🎬 Stage 1: Player Detection")

class SoccerPlayerDetector:
    """Detects players in a video using YOLOv8."""
    def __init__(self, model_name: str = 'yolov8n.pt', conf_thresh: float = 0.4):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model = YOLO(model_name)
        self.model.to(self.device)
        self.conf_thresh = conf_thresh
        print(f"Detector initialized on {self.device} with model {model_name}")

    def process_video(self, video_path: str, output_path: str) -> List[Dict]:
        """Processes a video file to detect players in each frame."""
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Error: Could not open video {video_path}")
            return []
        
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        all_detections = []
        
        with tqdm(total=total_frames, desc="Stage 1: Detecting players") as pbar:
            for frame_idx in range(total_frames):
                ret, frame = cap.read()
                if not ret:
                    break
                
                results = self.model(frame, classes=[0], verbose=False)
                
                detections = []
                if len(results) > 0 and results[0].boxes is not None:
                    for box in results[0].boxes:
                        if box.conf[0] >= self.conf_thresh:
                            detections.append({
                                'bbox': [int(coord) for coord in box.xyxy[0].tolist()],
                                'confidence': float(box.conf[0])
                            })
                
                all_detections.append({"frame_id": frame_idx, "detections": detections})
                pbar.update(1)
        
        cap.release()
        
        with open(output_path, 'w') as f:
            json.dump(all_detections, f, indent=2)
            
        return all_detections

if 'video_path' in locals() and video_path:
    detector = SoccerPlayerDetector()
    detections = detector.process_video(video_path, '/content/output/detections.json')
    print(f"✅ Detection complete! Processed {len(detections)} frames.")
else:
    print("❌ No video path available. Skipping detection.")

In [ ]:
# Stage 2: Multi-Object Tracking with a Robust ByteTrack Implementation
print("🎬 Stage 2: Multi-Object Tracking")

class KalmanFilter:
    """A simple Kalman Filter for tracking object motion."""
    def __init__(self):
        self.kf = cv2.KalmanFilter(7, 4)
        self.kf.measurementMatrix = np.array([[1,0,0,0,0,0,0], [0,1,0,0,0,0,0], [0,0,1,0,0,0,0], [0,0,0,1,0,0,0]], np.float32)
        self.kf.transitionMatrix = np.array([[1,0,0,0,1,0,0], [0,1,0,0,0,1,0], [0,0,1,0,0,0,0], [0,0,0,1,0,0,1], [0,0,0,0,1,0,0], [0,0,0,0,0,1,0], [0,0,0,0,0,0,1]], np.float32)
        cv2.setIdentity(self.kf.processNoiseCov, 1e-2)
        cv2.setIdentity(self.kf.measurementNoiseCov, 1e-1)
        cv2.setIdentity(self.kf.errorCovPost, 1)

    def predict(self) -> np.ndarray:
        return self.kf.predict()

    def update(self, bbox: List[float]):
        x, y, w, h = bbox
        measurement = np.array([x + w / 2, y + h / 2, w / h, h], dtype=np.float32).reshape(4, 1)
        self.kf.correct(measurement)

    def init(self, bbox: List[float]):
        x, y, w, h = bbox
        self.kf.statePost = np.array([x + w / 2, y + h / 2, w / h, h, 0, 0, 0], dtype=np.float32).reshape(7, 1)

class STrack:
    """Represents a single tracked object."""
    def __init__(self, tlwh: List[float], score: float):
        self.tlwh = np.asarray(tlwh, dtype=float)
        self.score = score
        self.kalman_filter = KalmanFilter()
        self.kalman_filter.init(self.tlwh)
        self.track_id = 0
        self.state = 'new'
        self.is_activated = False
        self.frame_id = 0
        self.start_frame = 0
        self.time_since_update = 0

    def activate(self, frame_id: int, track_id: int):
        self.track_id = track_id
        self.frame_id = frame_id
        self.start_frame = frame_id
        self.state = 'tracked'
        self.is_activated = True

    def re_activate(self, new_track, frame_id: int):
        self.tlwh = new_track.tlwh
        self.score = new_track.score
        self.kalman_filter.update(self.tlwh)
        self.state = 'tracked'
        self.is_activated = True
        self.frame_id = frame_id
        self.time_since_update = 0

    def predict(self):
        if self.state != 'tracked':
            self.kalman_filter.kf.statePost[6,0] = 0
        self.kalman_filter.predict()

    def update(self, new_track, frame_id: int):
        self.tlwh = new_track.tlwh
        self.score = new_track.score
        self.kalman_filter.update(self.tlwh)
        self.state = 'tracked'
        self.is_activated = True
        self.frame_id = frame_id
        self.time_since_update = 0

    @property
    def tlbr(self) -> List[float]:
        x, y, w, h = self.tlwh
        return [x, y, x + w, y + h]

def iou_distance(atracks: List[STrack], btracks: List[STrack]) -> np.ndarray:
    """Compute IOU distance between two sets of tracks."""
    if not atracks or not btracks: return np.empty((0, 0))
    atlbrs = np.array([track.tlbr for track in atracks])
    btlbrs = np.array([track.tlbr for track in btracks])
    ious = np.zeros((len(atlbrs), len(btlbrs)))
    for i, a in enumerate(atlbrs):
        for j, b in enumerate(btlbrs):
            box_inter = [max(a[0], b[0]), max(a[1], b[1]), min(a[2], b[2]), min(a[3], b[3])]
            inter_area = max(0, box_inter[2] - box_inter[0]) * max(0, box_inter[3] - box_inter[1])
            union_area = (a[2] - a[0]) * (a[3] - a[1]) + (b[2] - b[0]) * (b[3] - b[1]) - inter_area
            if union_area > 0: ious[i, j] = inter_area / union_area
    return 1 - ious

class ByteTrack:
    """The main ByteTrack algorithm implementation."""
    def __init__(self, high_thresh: float = 0.6, low_thresh: float = 0.1, max_time_lost: int = 30):
        self.tracked_stracks: List[STrack] = []
        self.lost_stracks: List[STrack] = []
        self.removed_stracks: List[STrack] = []
        self.frame_id = 0
        self.track_id_count = 0
        self.high_thresh = high_thresh
        self.low_thresh = low_thresh
        self.max_time_lost = max_time_lost

    def update(self, detections: List[Dict]) -> List[Dict]:
        self.frame_id += 1
        activated_starcks, refind_stracks, lost_stracks, removed_stracks = [], [], [], []
        dets_high = [d for d in detections if d['confidence'] >= self.high_thresh]
        dets_low = [d for d in detections if self.low_thresh <= d['confidence'] < self.high_thresh]
        stracks_high = [STrack([*d['bbox'][:2], d['bbox'][2]-d['bbox'][0], d['bbox'][3]-d['bbox'][1]], d['confidence']) for d in dets_high]
        stracks_low = [STrack([*d['bbox'][:2], d['bbox'][2]-d['bbox'][0], d['bbox'][3]-d['bbox'][1]], d['confidence']) for d in dets_low]

        for strack in self.tracked_stracks: strack.predict()
        dists = iou_distance(self.tracked_stracks, stracks_high)
        matches, u_track, u_detection = self.linear_assignment(dists, 0.8)
        for i, j in matches:
            track = self.tracked_stracks[i]
            det = stracks_high[j]
            track.update(det, self.frame_id)
            activated_starcks.append(track)

        unmatched_tracks = [self.tracked_stracks[i] for i in u_track]
        dists = iou_distance(unmatched_tracks, stracks_low)
        matches, u_track, u_detection_low = self.linear_assignment(dists, 0.5)
        for i, j in matches:
            track = unmatched_tracks[i]
            det = stracks_low[j]
            track.update(det, self.frame_id)
            activated_starcks.append(track)

        for i in u_track:
            track = unmatched_tracks[i]
            track.state = 'lost'
            lost_stracks.append(track)

        for i in u_detection:
            track = stracks_high[i]
            if track.score >= self.high_thresh:
                self.track_id_count += 1
                track.activate(self.frame_id, self.track_id_count)
                activated_starcks.append(track)

        self.tracked_stracks = [t for t in self.tracked_stracks if t.state == 'tracked'] + activated_starcks
        self.lost_stracks = [t for t in self.lost_stracks if t.time_since_update <= self.max_time_lost] + lost_stracks

        output = [{'track_id': t.track_id, 'bbox': [int(x) for x in t.tlbr]} for t in self.tracked_stracks if t.is_activated]
        return output

    def linear_assignment(self, cost_matrix, thresh):
        if cost_matrix.size == 0: return [], [], []
        row_ind, col_ind = linear_sum_assignment(cost_matrix)
        matches = [(r, c) for r, c in zip(row_ind, col_ind) if cost_matrix[r, c] < thresh]
        u_track = [r for r in range(cost_matrix.shape[0]) if r not in [m[0] for m in matches]]
        u_detection = [c for c in range(cost_matrix.shape[1]) if c not in [m[1] for m in matches]]
        return matches, u_track, u_detection

def run_tracking(detections_path: str, output_path: str) -> List[Dict]:
    with open(detections_path, 'r') as f: all_detections = json.load(f)
    tracker = ByteTrack()
    all_tracklets = []
    for frame_data in tqdm(all_detections, desc="Stage 2: Tracking players"):
        tracks = tracker.update(frame_data['detections'])
        all_tracklets.append({"frame_id": frame_data['frame_id'], "tracks": tracks})
    with open(output_path, 'w') as f: json.dump(all_tracklets, f, indent=2)
    return all_tracklets

tracklets = run_tracking('/content/output/detections.json', '/content/output/tracklets.json')
print(f"✅ Tracking complete! Processed {len(tracklets)} frames.")

In [ ]:
# Stage 3: Re-Identification with Deep Learning Features
print("🎬 Stage 3: Re-Identification")

class PlayerFeatureExtractor(nn.Module):
    """A simple CNN to extract deep features from player patches."""
    def __init__(self, embedding_dim=128):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, embedding_dim)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) # -> 16 x 32 x 32
        x = self.pool(F.relu(self.conv2(x))) # -> 32 x 16 x 16
        x = self.pool(F.relu(self.conv3(x))) # -> 64 x 8 x 8
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.normalize(x, p=2, dim=1)

class PlayerReID:
    """Manages long-term player identities using a hybrid feature model."""
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.feature_extractor = PlayerFeatureExtractor().to(self.device).eval()
        self.ocr = easyocr.Reader(['en'], gpu=torch.cuda.is_available())
        self.global_players: Dict[int, Dict] = {}
        self.next_permanent_id = 1
        self.similarity_threshold = 0.5
        self.jersey_bonus = 0.4

    def get_features(self, patch: np.ndarray) -> Dict:
        """Extracts color, deep, and jersey features from a player patch."""
        hsv = cv2.cvtColor(cv2.resize(patch, (64, 64)), cv2.COLOR_BGR2HSV)
        color_hist = cv2.normalize(cv2.calcHist([hsv], [0, 1], None, [30, 32], [0, 180, 0, 256]), None).flatten()
        img_tensor = torch.from_numpy(cv2.resize(patch, (64, 64))).permute(2, 0, 1).float().div(255).unsqueeze(0).to(self.device)
        with torch.no_grad():
            deep_features = self.feature_extractor(img_tensor).cpu().numpy().flatten()
        jersey = None
        try:
            gray = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)
            results = self.ocr.readtext(gray, allowlist='0123456789', detail=0, paragraph=False)
            if results and results[0].isdigit() and 1 <= len(results[0]) <= 2:
                jersey = int(results[0])
        except Exception:
            pass
        return {'color': color_hist, 'deep': deep_features, 'jersey': jersey}

    def process_tracklets(self, tracklets_path: str, video_path: str, output_path: str) -> List[Dict]:
        with open(tracklets_path, 'r') as f: all_tracklets = json.load(f)
        cap = cv2.VideoCapture(video_path)
        long_tracks = []
        
        for frame_data in tqdm(all_tracklets, desc="Stage 3: Re-identifying players"):
            frame_id = frame_data['frame_id']
            ret, frame = cap.read()
            if not ret: break
            
            frame_tracks = {"frame_id": frame_id, "players": []}
            for track in frame_data['tracks']:
                x1, y1, x2, y2 = track['bbox']
                patch = frame[y1:y2, x1:x2]
                if patch.size == 0: continue
                
                current_features = self.get_features(patch)
                best_id, best_score = None, self.similarity_threshold
                
                for pid, p_info in self.global_players.items():
                    color_sim = cv2.compareHist(current_features['color'], p_info['features']['color'], cv2.HISTCMP_CORREL)
                    deep_sim = cosine_similarity(current_features['deep'].reshape(1, -1), p_info['features']['deep'].reshape(1, -1))[0][0]
                    sim = 0.4 * color_sim + 0.6 * deep_sim
                    if current_features['jersey'] is not None and current_features['jersey'] == p_info['features']['jersey']:
                        sim += self.jersey_bonus
                    if sim > best_score:
                        best_score, best_id = sim, pid
                
                if best_id is None:
                    best_id = self.next_permanent_id
                    self.next_permanent_id += 1
                
                if best_id in self.global_players:
                    alpha = 0.1
                    self.global_players[best_id]['features']['color'] = (1-alpha) * self.global_players[best_id]['features']['color'] + alpha * current_features['color']
                    self.global_players[best_id]['features']['deep'] = (1-alpha) * self.global_players[best_id]['features']['deep'] + alpha * current_features['deep']
                else:
                    self.global_players[best_id] = {'features': current_features}
                
                frame_tracks["players"].append({"permanent_id": best_id, "bbox": track['bbox'], "jersey": current_features['jersey']})
            long_tracks.append(frame_tracks)
        
        cap.release()
        with open(output_path, 'w') as f: json.dump(long_tracks, f, indent=2)
        return long_tracks

reid = PlayerReID()
long_tracks = reid.process_tracklets('/content/output/tracklets.json', video_path, '/content/output/long_player_track.json')
print(f"✅ Re-ID complete! Identified {len(reid.global_players)} unique players.")

In [ ]:
# Stage 4: Intelligent Event Detection
print("🎬 Stage 4: Event Detection")

class SimpleEventDetector:
    """Detects events based on player kinematics and spatial locations."""
    def __init__(self, video_width: int, video_height: int):
        self.player_history: Dict[int, List] = {}
        self.goal_area = [video_width * 0.85, video_height * 0.2, video_width, video_height * 0.8]
        self.velocity_threshold = 15.0
        self.cluster_threshold = 100
        self.cluster_size_threshold = 5

    def detect_events(self, player_tracks_path: str, video_path: str, output_path: str) -> List[Dict]:
        with open(player_tracks_path, 'r') as f: all_tracks = json.load(f)
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        events = []

        for frame_data in tqdm(all_tracks, desc="Stage 4: Detecting events"):
            frame_id = frame_data['frame_id']
            current_players = frame_data['players']
            player_centers = {}

            for p in current_players:
                pid = p['permanent_id']
                x1, y1, x2, y2 = p['bbox']
                center = ((x1+x2)/2, (y1+y2)/2)
                player_centers[pid] = center
                
                if pid in self.player_history and len(self.player_history[pid]) > 1:
                    prev_center = self.player_history[pid][-2]['center']
                    velocity = math.hypot(center[0] - prev_center[0], center[1] - prev_center[1])
                    if velocity > self.velocity_threshold:
                        events.append({'frame_id': frame_id, 'timestamp': frame_id / fps, 'event_type': 'sprint', 'player_id': pid})
                
                if self.goal_area[0] < center[0] < self.goal_area[2] and self.goal_area[1] < center[1] < self.goal_area[3]:
                    events.append({'frame_id': frame_id, 'timestamp': frame_id / fps, 'event_type': 'goal_area_entry', 'player_id': pid})

                if pid not in self.player_history: self.player_history[pid] = []
                self.player_history[pid].append({'frame': frame_id, 'center': center})

            if len(player_centers) > self.cluster_size_threshold:
                from sklearn.cluster import DBSCAN
                clustering = DBSCAN(eps=self.cluster_threshold, min_samples=self.cluster_size_threshold).fit(list(player_centers.values()))
                if len(set(clustering.labels_)) > 1:
                     events.append({'frame_id': frame_id, 'timestamp': frame_id / fps, 'event_type': 'cluster_action'})

        cap.release()
        unique_events = []
        seen_frames = set()
        for event in sorted(events, key=lambda x: x['frame_id']):
            if event['frame_id'] not in seen_frames:
                unique_events.append(event)
                seen_frames.add(event['frame_id'])
        
        with open(output_path, 'w') as f: json.dump(unique_events, f, indent=2)
        return unique_events

def filter_player_events(events_path: str, target_player_id: int, output_path: str) -> List[Dict]:
    with open(events_path, 'r') as f: all_events = json.load(f)
    player_events = [e for e in all_events if e.get('player_id') == target_player_id or e['event_type'] == 'cluster_action']
    with open(output_path, 'w') as f: json.dump(player_events, f, indent=2)
    return player_events

if 'video_path' in locals() and video_path:
    cap = cv2.VideoCapture(video_path)
    video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()

    event_detector = SimpleEventDetector(video_width, video_height)
    events = event_detector.detect_events('/content/output/long_player_track.json', video_path, '/content/output/events.json')

    target_player_id = 1
    player_events = filter_player_events('/content/output/events.json', target_player_id, '/content/output/player_events.json')
    print(f"✅ Event detection complete! Found {len(events)} total potential events, {len(player_events)} for player {target_player_id}")

In [ ]:
# Stage 5: Professional Video Assembly with PySceneDetect
print("🎬 Stage 5: Video Assembly & Highlight Generation")

class VideoAssembler:
    """Assembles the final highlight reel using intelligent scene cutting."""
    def __init__(self, clip_padding_seconds: float = 1.0, max_reel_duration: float = 300.0):
        self.clip_padding = clip_padding_seconds
        self.max_duration = max_reel_duration
        self.temp_dir = '/content/temp_clips'

    def find_scenes(self, video_path: str) -> List[Tuple[float, float]]:
        """Use PySceneDetect to find all scene boundaries in the video."""
        video = open_video(video_path)
        scene_manager = SceneManager()
        scene_manager.add_detector(ContentDetector(threshold=27.0))
        scene_manager.detect_scenes(video, show_progress=True)
        scene_list = scene_manager.get_scene_list()
        return [(s[0].get_seconds(), s[1].get_seconds()) for s in scene_list]

    def assemble_highlight_reel(self, video_path: str, player_events_path: str, output_path: str) -> bool:
        with open(player_events_path, 'r') as f: player_events = json.load(f)
        if not player_events: 
            print("No events for target player. Cannot create reel.")
            return False
        
        print("Finding scene cuts... (this may take a moment)")
        scenes = self.find_scenes(video_path)
        event_timestamps = sorted([e['timestamp'] for e in player_events])
        
        clips_to_extract = []
        total_duration = 0.0
        for ts in event_timestamps:
            if total_duration >= self.max_duration: break
            for start, end in scenes:
                if start <= ts <= end:
                    clip_duration = end - start
                    if total_duration + clip_duration <= self.max_duration:
                        if not any(c['start'] == start for c in clips_to_extract):
                            clips_to_extract.append({'start': start, 'end': end})
                            total_duration += clip_duration
                    break
        
        if not clips_to_extract: 
            print("Could not map any events to scenes.")
            return False
        
        os.makedirs(self.temp_dir, exist_ok=True)
        clip_paths = []
        for i, clip in enumerate(tqdm(clips_to_extract, desc="Extracting scene clips")):
            clip_path = os.path.join(self.temp_dir, f"clip_{i:04d}.mp4")
            command = ['ffmpeg', '-y', '-i', video_path, '-ss', str(clip['start']), '-to', str(clip['end']), '-c', 'copy', '-avoid_negative_ts', '1', clip_path]
            result = subprocess.run(command, capture_output=True, text=True)
            if result.returncode == 0: clip_paths.append(clip_path)
        
        concat_list_path = os.path.join(self.temp_dir, "concat_list.txt")
        with open(concat_list_path, 'w') as f:
            for path in clip_paths:
                f.write(f"file '{os.path.basename(path)}'\n")
        
        concat_command = ['ffmpeg', '-y', '-f', 'concat', '-safe', '0', '-i', concat_list_path, '-c', 'copy', output_path]
        concat_result = subprocess.run(concat_command, cwd=self.temp_dir, capture_output=True, text=True)

        if concat_result.returncode == 0:
            print(f"Highlight reel created: {output_path}")
            print(f"Total duration: {total_duration:.2f} seconds")
            return True
        else:
            print("Failed to concatenate clips. FFmpeg error:", concat_result.stderr)
            return False

if 'video_path' in locals() and video_path:
    assembler = VideoAssembler()
    highlight_path = f'/content/output/player_{target_player_id}_highlights.mp4'
    success = assembler.assemble_highlight_reel(video_path, '/content/output/player_events.json', highlight_path)

    if success:
        print(f"✅ Highlight reel saved to: {highlight_path}")
    else:
        print("❌ Failed to create highlight reel")

In [ ]:
# 4. Download Results
print("📥 Downloading Results")

from google.colab import files

if 'success' in locals() and success and 'highlight_path' in locals() and os.path.exists(highlight_path):
    files.download(highlight_path)
    print(f"✅ Downloaded: {os.path.basename(highlight_path)}")
else:
    print(f"❌ Could not download highlight reel. File not found or creation failed.")

if os.path.exists('/content/output/long_player_track.json'):
    files.download('/content/output/long_player_track.json')
    print("✅ Downloaded: long_player_track.json")

if os.path.exists('/content/output/player_events.json'):
    files.download('/content/output/player_events.json')
    print("✅ Downloaded: player_events.json")

print("\n🎉 Pipeline complete! Check your browser's downloads folder.")

In [ ]:
# 5. Pipeline Summary
print("📊 PIPELINE SUMMARY")
print("=" * 50)

if 'video_path' in locals() and video_path and os.path.exists('/content/output/long_player_track.json') and os.path.exists('/content/output/player_events.json'):
    with open('/content/output/long_player_track.json', 'r') as f:
        long_tracks_summary = json.load(f)
    with open('/content/output/player_events.json', 'r') as f:
        player_events_summary = json.load(f)

    print(f"📹 Video processed: {os.path.basename(video_path)}")
    print(f"🎯 Target player ID: {target_player_id}")
    print(f"🖼️ Total frames processed: {len(long_tracks_summary)}")
    print(f"✨ Events detected for player: {len(player_events_summary)}")

    if 'success' in locals() and success and 'highlight_path' in locals() and os.path.exists(highlight_path):
        print(f"⏱️ Highlight reel generated successfully.")
        file_size = os.path.getsize(highlight_path) / (1024 * 1024)
        print(f"📁 Output file size: {file_size:.1f} MB")
    else:
        print("⏱️ Highlight reel was not generated.")
else:
    print("Could not generate summary. One or more output files are missing.")

print("=" * 50)
print("🎉 Pipeline finished!")